# Lab: pycos

Write a pycos datetime server that will send the client the current `time.ctime()` every 10s

In [1]:
import time
import socket
import pycos

def handler_proc(sock, task=None):
    print('In handler')
    task.set_daemon()
    while True:
        try:
            result = yield sock.send(time.ctime().encode('utf-8') + b'\n')
        except Exception as err:
            print(err)
            break
        yield task.sleep(10)
    print('Leaving handler')
    sock.close()
        
def server_proc(sock, host, port, task=None):
    task.set_daemon()
    sock = pycos.AsyncSocket(sock)
    sock.bind((host, port))
    sock.listen(5)

    while True:
        conn, addr = yield sock.accept()
        pycos.Task(handler_proc, conn)
        
sock = socket.socket()
server = pycos.Task(server_proc, sock, '127.0.0.1', 8001)
server

2020-05-22 15:16:13 pycos - version 4.8.15 with kqueue I/O notifier


!server_proc/4376341584

In handler


In [2]:
sock.close()
server.terminate()

0

[Errno 32] Broken pipe
Leaving handler


# Lab: netpycos

Using the "Distributed Channels" example in the demo, 

- write a timeserver that will broadcast the current `time.ctime()` every 1s to a named channel
- write a listener that will print the values received from the timeserver

In [3]:
%%file data/pycos-examples/timeserver.py
import sys
import time
import pycos
import pycos.netpycos

from my_ip import MY_IP

sched = pycos.Pycos.instance(node=MY_IP)
# sched = pycos.Pycos.instance()
channel = pycos.Channel('timeserver')

def server_proc(task=None):
    task.set_daemon()
    channel.register()
    while True:
        channel.send(time.ctime())
        yield task.sleep(1)

server = pycos.Task(server_proc)
while True:
    cmd = sys.stdin.readline().strip().lower()
    if cmd == 'quit' or cmd == 'exit':
        break

Overwriting data/pycos-examples/timeserver.py


In [4]:
%%file data/pycos-examples/time_listener.py
import sys
import pycos
import pycos.netpycos

from my_ip import MY_IP

sched = pycos.Pycos.instance(node=MY_IP) #, udp_port=9706, tcp_port=9706)
# sched = pycos.Pycos.instance()

def main_proc(task=None):
    task.set_daemon()
    chan = yield pycos.Channel.locate('timeserver')
    print(f'Located {chan}')
    yield chan.subscribe(task)
    while True:
        msg = yield task.receive()
        print(f'received {msg}')
    
pycos.Task(main_proc)
while True:
    cmd = sys.stdin.readline().strip().lower()
    if cmd == 'quit' or cmd == 'exit':
        break


Overwriting data/pycos-examples/time_listener.py
